In [1]:
import numpy as np
import sys
from time import time

In [2]:
class Node:
    def __init__(self, parent, state, pcost, hcost):
        # Initialize node with parent node, state, path cost, and heuristic cost
        self.parent = parent
        self.state = state
        self.pcost = pcost
        self.hcost = hcost
        self.cost = pcost + hcost  # Total cost = path cost + heuristic cost
    
    def __hash__(self):
        # Hash function to generate hash value for the node's state
        return hash(''.join(self.state.flatten()))  # Convert state to a hashable string
    
    def __str__(self):
        # String representation of the node's state
        return str(self.state)
    
    def __eq__(self, other):
        # Check if two nodes are equal
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten()))  # Compare hash values
    
    def __ne__(self, other):
        # Check if two nodes are not equal
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))  # Compare hash values


In [3]:
class PriorityQueue():
    
    def __init__(self):
        # Initialize priority queue as an empty list
        self.queue = []
        
    def push(self, node):
        # Add a node to the priority queue
        self.queue.append(node)
    
    def pop(self):
        # Remove and return the node with the least cost from the priority queue
        
        # Initialize variables to keep track of the node with the least cost
        next_state = None
        state_cost = 10**18  # Initialize with a large number
        index = -1  # Initialize index of node with least cost
        
        # Iterate through the queue to find the node with the least cost
        for i in range(len(self.queue)):
            if self.queue[i].cost < state_cost:
                state_cost = self.queue[i].cost
                index = i
        
        # Remove and return the node with the least cost
        return self.queue.pop(index)
    
    def is_empty(self):
        # Check if the priority queue is empty
        return len(self.queue) == 0
    
    def __str__(self):
        # String representation of the priority queue
        l = []
        for i in self.queue:
            l.append(i.state)  # Append state of each node to the list
        return str(l)
    
    def __len__(self):
        # Get the length of the priority queue
        return len(self.queue)


In [4]:
import numpy as np

class Environment():
    
    def __init__(self, depth=None, goal_state=None):
        # Define actions (1: Up, 2: Down, 3: Right, 4: Left)
        self.actions = [1, 2, 3, 4]
        self.goal_state = goal_state  # Define goal state of the environment
        self.depth = depth  # Define maximum depth of the solution
        self.start_state = self.generate_start_state()  # Generate start state
    
    def generate_start_state(self):
        # Generate start state by performing 'depth' random moves from the goal state
        
        past_state = self.goal_state
        i = 0
        while i != self.depth:
            new_states = self.get_next_states(past_state)
            choice = np.random.randint(low=0, high=len(new_states))
            
            if np.array_equal(new_states[choice], past_state):
                continue
            
            past_state = new_states[choice]
            i += 1
            
        return past_state
    
    def get_start_state(self):
        # Return the start state
        return self.start_state
    
    def get_goal_state(self):
        # Return the goal state
        return self.goal_state
    
    def get_next_states(self, state):
        # Given a state, return all possible next states
        
        space = (0, 0)
        for i in range(3):
            for j in range(3):
                if state[i, j] == '_':
                    space = (i, j)
                    break
        
        new_states = []
        
        if space[0] > 0:  # Move Up
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0] - 1, space[1]]
            new_state[space[0] - 1, space[1]] = val
            
            new_states.append(new_state)
            
        if space[0] < 2:  # Move Down
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0] + 1, space[1]]
            new_state[space[0] + 1, space[1]] = val
            
            new_states.append(new_state)
        
        if space[1] < 2:  # Move Right
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1] + 1]
            new_state[space[0], space[1] + 1] = val
            
            new_states.append(new_state)
            
        if space[1] > 0:  # Move Left
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1] - 1]
            new_state[space[0], space[1] - 1] = val
            
            new_states.append(new_state)
        
        return new_states
    
    def reached_goal(self, state):
        # Check if the given state is the goal state
        
        for i in range(3):
            for j in range(3):
                if state[i, j] != self.goal_state[i, j]:
                    return False
        
        return True


In [5]:
class Agent:
    
    def __init__(self, env, heuristic):
        # Initialize agent with priority queue for frontier, dictionary for explored nodes,
        # start state, goal state, environment, goal node, and heuristic function
        self.frontier = PriorityQueue()
        self.explored = dict()
        self.start_state = env.get_start_state()
        self.goal_state = env.get_goal_state()
        self.env = env
        self.goal_node = None
        self.heuristic = heuristic
    
    def run(self):
        # Explore the environment and find the goal node
        
        # Initialize the initial node with start state
        init_node = Node(parent=None, state=self.start_state, pcost=0, hcost=0)
        self.frontier.push(init_node)
        steps = 0
        while not self.frontier.is_empty():

            curr_node = self.frontier.pop()
            next_states = self.env.get_next_states(curr_node.state)

            if hash(curr_node) in self.explored:
                continue

            self.explored[hash(curr_node)] = curr_node

            if self.env.reached_goal(curr_node.state):
                self.goal_node = curr_node
                break
            
            goal_state = self.env.get_goal_state()

            for state in next_states:
                # Calculate heuristic cost for next states
                hcost = self.heuristic(state, goal_state)
                node = Node(parent=curr_node, state=state, pcost=curr_node.pcost + 1, hcost=hcost)
                self.frontier.push(node)
            steps += 1
        
        return steps, self.soln_depth()
    
    def soln_depth(self):
        # Calculate solution depth
        node = self.goal_node
        count = 0
        while node is not None:
            node = node.parent
            count += 1
        
        return count
    
    def print_nodes(self):
        # Print the path from start node to goal node
        
        node = self.goal_node
        path = []
        while node is not None:
            path.append(node)
            node = node.parent

        step = 1
        for node in path[::-1]:
            print("Step:", step)
            print(node)
            step += 1
    
    def get_memory(self):
        # Get memory usage
        
        # Estimate memory usage based on the size of frontier and explored nodes
        mem = len(self.frontier) * 56 + len(self.explored) * 56
        return mem


In [8]:
def heuristic2(curr_state, goal_state):
    # Heuristic function that calculates the Manhattan distance between the current state and the goal state
    
    dist = 0

    for i in range(3):
        for j in range(3):
            ele = curr_state[i, j]  # Current marble
            goal_i, goal_j = np.where(goal_state == ele)  # Find position of the same marble in the goal state
            d = abs(goal_i[0] - i) + abs(goal_j[0] - j)  # Calculate Manhattan distance
            dist += d
    
    return dist


In [9]:
depth = 500
goal_state = np.array([[1,2,3], [8,'_',4], [7,6,5]])
env = Environment(depth, goal_state)
print("Start State: ")
print(env.get_start_state())
print("Goal State: ")
print(goal_state)

Start State: 
[['2' '8' '3']
 ['6' '_' '4']
 ['5' '1' '7']]
Goal State: 
[['1' '2' '3']
 ['8' '_' '4']
 ['7' '6' '5']]


In [10]:
agent = Agent(env = env, heuristic = heuristic2)

In [11]:
agent.run()

(284, 19)

In [12]:

depths = np.arange(0, 501, 50)
goal_state = np.array([[1, 2, 3], [8, '_', 4], [7, 6, 5]])
times_taken = {}
mems = {}

# Iterate over different depths
for depth in depths:
    time_taken = 0
    mem = 0
    
    # Repeat each experiment 50 times
    for i in range(50):
        # Create environment and agent
        env = Environment(depth=depth, goal_state=goal_state)
        agent = Agent(env=env, heuristic=heuristic2)
        
        # Measure time taken
        start_time = time()
        agent.run()
        end_time = time()
        time_taken += end_time - start_time
        
        # Measure memory used
        mem += agent.get_memory()
    
    # Average time taken and memory used
    time_taken /= 50
    mem /= 50
    
    # Store results
    times_taken[depth] = time_taken
    mems[depth] = mem
    
    # Print results
    print(depth, time_taken, mem)


0 0.0001175832748413086 56.0
50 0.062335538864135745 15208.48
100 0.17607250690460205 37699.2
150 0.8281965398788452 103038.88
200 0.7005016040802002 100705.92
250 0.5602542448043824 91827.68
300 1.707741904258728 175056.0
350 0.7300944328308105 110492.48
400 1.3042884254455567 135346.4
450 1.0080248785018922 121304.96
500 1.1396568250656127 125048.0
